In [41]:
# @title TZ0C LAB v4.0
# =======================================================
# // --- (Master Cell id: 6GASOcD0_Run) --- //
# =======================================================
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from IPython.display import display, clear_output, Markdown, HTML, Math
import ipywidgets as widgets
from google.colab import files, output, drive
import plotly.graph_objects as go
import scipy.stats as stats # New import for statistical functions
import uuid # New import for unique identifiers
from datetime import datetime # New import for timestamps

# =======================================================
# //--- THEME & CONSTANTS ---//
# =======================================================
try:
    is_dark = output.eval_js('document.documentElement.getAttribute("theme") == "dark"')
except:
    is_dark = False

plt_style = 'dark_background' if is_dark else 'default'
text_color = 'white' if is_dark else 'black'
accent_color = '#00e6e6' if is_dark else '#008b8b'

TETRAHEDRAL_ANGLE = np.degrees(np.arccos(-1/3))
print(f"✅ Exact tetrahedral angle: {TETRAHEDRAL_ANGLE:.10f}°")

# =======================================================
# // --- GLOBAL CONSTANTS AND HELPER FUNCTIONS --- //
# =======================================================
F0 = 1.62e14
# Global targets for molecular calculations, not element-specific
targets = {"H2O": 104.5, "Peptide": 107.2, "DNA_Base": 108.4}
# Global weights for molecular calculations, not element-specific
weights = {"C": 12.01, "O": 16.00, "H": 1.008, "N": 14.01}
C_SQUARED_C_CUBED_FLIP_FREQ_THZ = 162.1 # New global constant as per instructions

# Helper to create elements data from material_registry if needed
def create_mock_elements_from_material_registry(registry):
    mock_elements = {}
    for name, props in registry.items():
        symbol_match = re.search(r'\\((?:\\w+)\\)', name)

        if symbol_match:
            symbol = symbol_match.group(1)
            element_name = name.split(' (')[0]
        else:
            symbol = name.split(' ')[0]
            element_name = name

        mock_elements[symbol] = {
            'invariant_angle': props.get('node', TETRAHEDRAL_ANGLE),
            'shake_hz': props.get('f0', F0),
            'name': element_name,
            'atomic_number': 0, # Placeholder
            'flip_risk': 0.5,   # Placeholder
            'torque_density_alpha': 1.0 # Placeholder
        }
    return mock_elements

# New helper function to extract elements list from JSON
def _extract_elements_list_from_json(json_data):
    if 'elements' in json_data:
        if isinstance(json_data['elements'], list):
            return json_data['elements']
        elif isinstance(json_data['elements'], dict):
            return list(json_data['elements'].values()) # Extract values if 'elements' is a dict
    if 'registry' in json_data and isinstance(json_data['registry'], list):
        return json_data['registry']
    if 'materials' in json_data and isinstance(json_data['materials'], list):
        return json_data['materials']
    if isinstance(json_data, list):
        return json_data
    return None

# =======================================================
# // --- UNIFIED DATA LOADING --- //
# =======================================================
unified_json_data_path = '/content/drive/MyDrive/Research_Journal/TZ0C_Lab_v4a.json'
unified_json_data = {}

try:
    drive.mount('/content/drive', force_remount=True)
    if os.path.exists(unified_json_data_path):
        with open(unified_json_data_path, 'r') as f:
            unified_json_data = json.load(f)
        print(f"✅ Loaded unified JSON data from {unified_json_data_path}")
    else:
        unified_json_data = {}
        print(f"⚠️ Unified JSON file not found at {unified_json_data_path}. Initializing empty unified_json_data.")
except Exception as e:
    unified_json_data = {}
    print(f"⚠️ Unified JSON loading error: {e}. Initializing empty unified_json_data.")

# =======================================================
# // --- DYNAMICALLY UPDATE TARGETS DICTIONARY --- //
# =======================================================
if 'molecular_analysis_results' in unified_json_data:
    for mol_name, results in unified_json_data['molecular_analysis_results'].items():
        if 'best_spot_angle' in results:
            targets[mol_name] = results['best_spot_angle']
    print("✅ Dynamically updated 'targets' dictionary with molecular best spot angles.")

# =======================================================
# // --- MATERIAL REGISTRY (118 elements) --- //
# =======================================================
material_registry = {}
try:
    registry_data_source = unified_json_data.get('registry_data', {})
    json_data_for_loop = _extract_elements_list_from_json(registry_data_source)

    if json_data_for_loop:
        for item in json_data_for_loop:
            name = f"{item.get('name', 'Unknown')} ({item.get('symbol', 'UKN')})"
            material_registry[name] = {
                "mass": float(item.get('atomic_weight', item.get('mass', 0.0))),
                # Prioritize _absolute_invariant_angle for node
                "node": float(item.get('_absolute_invariant_angle', item.get('perfect_angle', item.get('invariant_angle', TETRAHEDRAL_ANGLE)))),
                "f0": float(item.get('shake_hz', F0)), # Keep shake_hz for f0 as _shake_hz_calibrated is not available
                "color": "magenta" if item.get('atomic_number', 0) == 85 else "cyan"
            }
        print(f"✅ Populated material registry from unified JSON data ({len(material_registry)} entries)")
    else:
        print("⚠️ No element list found in unified JSON 'registry_data'. Material registry remains empty.")
except Exception as e:
    print(f"⚠️ Material registry population error from unified JSON: {e}. Material registry remains empty.")

current_material = list(material_registry.keys())[0] if material_registry else "No Material"

def on_material_change(change):
    global current_material
    current_material = change['new']
    if current_material in material_registry:
        prof = material_registry[current_material]
        output_manager.display_output(
            f"🔄 Target changed → {current_material} | Mass: {prof['mass']} u | Node: {prof['node']:.6f}° | f₀: {prof['f0']/1e12:.1f} THz",
            content_type='text'
        )
    else:
        output_manager.display_output(f"🔄 Selected material '{current_material}' not found in registry.", content_type='text')

# =======================================================
# // --- DATA & MARKDOWN --- //
# =======================================================
def load_data():
    global unified_json_data

    if 'baseline_simulation_data' in unified_json_data and unified_json_data['baseline_simulation_data']:
        try:
            df_sim = pd.DataFrame(unified_json_data['baseline_simulation_data'])
            print("✅ Loaded baseline simulation data from unified_json_data.")
            return df_sim
        except Exception as e:
            print(f"⚠️ Error parsing baseline simulation data from unified_json_data: {e}. Returning empty DataFrame.")
            print("Please generate basic data using the 'Raw Data Generation' tab if no data is displayed.")
            return pd.DataFrame()
    else:
        print("⚠️ 'baseline_simulation_data' not found in unified_json_data or is empty. Generating fallback data.")

        synthetic_records = []
        molecular_results = unified_json_data.get('molecular_analysis_results', {})
        element_results = unified_json_data.get('element_invariant_results', {})

        # Add data points centered around global tetrahedral angle
        for _ in range(3):
            synthetic_records.append({
                'theta': np.random.normal(TETRAHEDRAL_ANGLE, 0.01),
                'freq': np.random.normal(F0, 1e11),
                'lock': np.random.uniform(0.1, 1.0),
                'wave': np.random.uniform(0.1, 1.0)
            })

        # Add data points from H2O molecular analysis if available
        if 'H2O' in molecular_results and 'best_spot_angle' in molecular_results['H2O']:
            for _ in range(2):
                synthetic_records.append({
                    'theta': np.random.normal(molecular_results['H2O']['best_spot_angle'], 0.005),
                    'freq': np.random.normal(F0, 1e11),
                    'lock': np.random.uniform(0.1, 1.0),
                    'wave': np.random.uniform(0.1, 1.0)
                })

        # Add data points from Peptide molecular analysis if available
        if 'Peptide' in molecular_results and 'best_spot_angle' in molecular_results['Peptide']:
            for _ in range(2):
                synthetic_records.append({
                    'theta': np.random.normal(molecular_results['Peptide']['best_spot_angle'], 0.005),
                    'freq': np.random.normal(F0, 1e11),
                    'lock': np.random.uniform(0.1, 1.0),
                    'wave': np.random.uniform(0.1, 1.0)
                })

        # Add a couple of points based on a sample element's absolute invariant angle if available
        if element_results:
            # Try to get H's angle if available
            h_angle = element_results.get('H', {}).get('absolute_invariant_angle')
            if h_angle:
                synthetic_records.append({
                    'theta': np.random.normal(h_angle, 0.02),
                    'freq': np.random.normal(F0, 1e11),
                    'lock': np.random.uniform(0.1, 1.0),
                    'wave': np.random.uniform(0.1, 1.0)
                })
            # Try to get C's angle if available
            c_angle = element_results.get('C', {}).get('absolute_invariant_angle')
            if c_angle:
                synthetic_records.append({
                    'theta': np.random.normal(c_angle, 0.01),
                    'freq': np.random.normal(F0, 1e11),
                    'lock': np.random.uniform(0.1, 1.0),
                    'wave': np.random.uniform(0.1, 1.0)
                })

        df_fallback = pd.DataFrame(synthetic_records)
        print("✅ Using fallback data for baseline simulation to enable plots.")
        return df_fallback

df = load_data()

tz0c_sections = unified_json_data.get('tz0c_markdown_sections', {})
if not tz0c_sections:
    print("⚠️ 'tz0c_markdown_sections' not found in unified JSON. Displaying empty content for theory sections.")
else:
    print("✅ Populated TZ0C sections from unified JSON data.")

# =======================================================
# // --- OutputManager Class --- //
# =======================================================
class OutputManager:
    def __init__(self, max_outputs=3):
        self.max_outputs = max_outputs
        self.outputs = []
        self.vbox_container = widgets.VBox([])
        self.visible_output_slots_used = 0

    def _create_output_container(self, content_widget, content_type, content_raw):
        btn_print = widgets.Button(description='Print', button_style='info')
        btn_save = widgets.Button(description='Save', button_style='success')
        btn_share = widgets.Button(description='Share', button_style='warning')
        btn_close = widgets.Button(description='Close', button_style='danger')

        def on_print_click(b): print(f"Printing {content_type} output...")
        def on_save_click(b): print(f"Saving {content_type} output...")
        def on_share_click(b): print(f"Sharing {content_type} output...")

        container = widgets.VBox([
            widgets.HBox([widgets.Label(value=f"Output ({content_type})"), btn_print, btn_save, btn_share, btn_close]),
            content_widget
        ], layout=widgets.Layout(border='2px solid lightgray', margin='5px', padding='5px'))

        def on_close_click(b):
            self.vbox_container.children = tuple(c for c in self.vbox_container.children if c is not container)

            new_outputs = []
            for output_entry in self.outputs:
                if output_entry[2] is container:
                    self.visible_output_slots_used -= output_entry[3]
                else:
                    new_outputs.append(output_entry)
            self.outputs = new_outputs

        btn_print.on_click(on_print_click)
        btn_save.on_click(on_save_click)
        btn_share.on_click(on_share_click)
        btn_close.on_click(on_close_click)

        return container

    def display_output(self, content, content_type='text'):
        slots_needed = 1
        if content_type in ['text', 'markdown', 'latex'] and len(str(content)) > 1000:
            slots_needed = self.max_outputs

        while self.visible_output_slots_used + slots_needed > self.max_outputs:
            if not self.outputs:
                print("OutputManager: Warning! No outputs to remove, but new output exceeds max_outputs.")
                break

            old_content_type, old_content_raw, old_container_widget, old_slots_used = self.outputs.pop(0)

            self.vbox_container.children = tuple(c for c in self.vbox_container.children if c is not old_container_widget)
            self.visible_output_slots_used -= old_slots_used

        output_capture_area = widgets.Output()

        with output_capture_area:
            if content_type == 'text':
                if isinstance(content, str):
                    print(content)
                else:
                    display(content)
            elif content_type == 'markdown':
                display(Markdown(content))
            elif content_type == 'html':
                display(HTML(content))
            elif content_type == 'math':
                display(Math(content))
            elif content_type == 'matplotlib_figure':
                if isinstance(content, plt.Figure):
                    display(content)
                    plt.close(content)
                else:
                    content()

            elif content_type == 'plotly_figure':
                if isinstance(content, go.Figure):
                    display(content)
                else:
                    content()
            else:
                print(f"OutputManager: Unsupported content_type '{content_type}'. Displaying as text.\n{content}")

        new_container_widget = self._create_output_container(output_capture_area, content_type, content)

        self.outputs.append((content_type, content, new_container_widget, slots_needed))
        self.vbox_container.children = (*self.vbox_container.children, new_container_widget)
        self.visible_output_slots_used += slots_needed

output_manager = OutputManager(max_outputs=3)

# =======================================================
# // --- PLOTS & SIMS --- //
# =======================================================
def apply_style(title, xl=None, yl=None):
    plt.style.use(plt_style)
    plt.title(title, color=text_color, fontsize=14, fontweight='bold')
    if xl: plt.xlabel(xl, color=text_color)
    if yl: plt.ylabel(yl, color=text_color)
    plt.grid(True, alpha=0.2)

def plot_dual_sweep(_):
    if df.empty:
        output_manager.display_output("❌ No data loaded. Please generate basic data in 'Raw Data Generation' tab.", content_type='text')
        return
    fig = plt.figure(figsize=(10,6))
    plt.hist(df['theta'], bins=100, density=True, alpha=0.3, color='gray')
    plt.axvline(109.33,color='orange',ls='--',lw=2,label="Wave 109.33°")
    plt.axvline(109.47,color=accent_color,ls='--',lw=2,label="Lock 109.47°")
    x=np.linspace(109.2,109.6,1000)
    plt.plot(x,np.exp(-(x-109.33)**2/(2*0.02**2))*0.5,color='orange',lw=3)
    plt.plot(x,np.exp(-(x-109.47)**2/(2*0.02**2))*0.5,color=accent_color,lw=3)
    apply_style("0.14° Geometric Bounce Gap","Theta (°)","Density")
    plt.legend()
    output_manager.display_output(fig, content_type='matplotlib_figure')

def plot_handshake(_):
    if df.empty:
        output_manager.display_output("❌ No data loaded. Please generate basic data in 'Raw Data Generation' tab.", content_type='text')
        return
    fig = plt.figure(figsize=(10,5))
    apply_style("162 THz Handshake","Freq (THz)","Lock/Wave Ratio")
    plt.scatter(df['freq']/1e12, df['lock']/df['wave'], c=df['theta'], cmap='magma' if is_dark else 'viridis')
    plt.axvline(C_SQUARED_C_CUBED_FLIP_FREQ_THZ,color='red',lw=2,label="c²/c³ Flip")
    plt.colorbar(label='Theta')
    plt.legend()
    output_manager.display_output(fig, content_type='matplotlib_figure')

def plot_element_potential_or_stability(_):
    if not current_material or current_material not in material_registry:
        output_manager.display_output("❌ No material selected or material not found in registry.", content_type='text')
        return
    fig = plt.figure(figsize=(10,5))
    r=np.linspace(0.1,2,500)
    prof=material_registry[current_material]
    m=prof['mass']
    # MODIFICATION 3: Use element-specific f0 from prof instead of global F0
    omega=prof['f0']*2*np.pi
    stability_metric=(3*m*omega**4)/(4*np.pi*r)
    plt.plot(r,np.log10(stability_metric),color=prof['color'],lw=3)
    apply_style(f"Element Stability: {current_material}","Radius (Å)","log10(Stability Metric)")
    output_manager.display_output(fig, content_type='matplotlib_figure')

def run_diamond_core_sim(cycles=10000):
    if not current_material or current_material not in material_registry:
        output_manager.display_output("❌ No material selected or material not found in registry.", content_type='text')
        return
    prof = material_registry[current_material]
    mu = prof['node']
    log_content = [f"🚀 Running {current_material} Core Sim — {cycles:,} cycles"]
    theta_history = []

    for i in range(1,cycles+1):
        current_theta = mu + np.random.normal(0,0.001)
        theta_history.append(current_theta)

        if i % max(20000,cycles//5)==0 or i==cycles:
            log_content.append(f"   Cycle {i:6,} → θ = {current_theta:.6f}°")
    log_content.append(f"🏁 {current_material} Rigidity Lock achieved at {mu:.6f}°")
    output_manager.display_output('\n'.join(log_content), content_type='text')

    fig = plt.figure(figsize=(10, 6))
    plt.plot(range(1, cycles + 1), theta_history, color=prof['color'], linewidth=1)
    apply_style(
        f"Diamond Core Simulation: {current_material} Invariant Angle Variation",
        "Cycle",
        "Invariant Angle (°)"
    )
    output_manager.display_output(fig, content_type='matplotlib_figure')

def run_janibekov_audit(_):
    if not current_material or current_material not in material_registry:
        output_manager.display_output("❌ No material selected or material not found in registry.", content_type='text')
        return
    prof = material_registry[current_material]
    t_flip = prof['f0'] / 2.5285

    html_heading = f"<h3 style='color:{accent_color}'>C-Ladder Audit: {current_material}</h3>"
    text_summary = f"Janibekov T-Flip: {t_flip:.2e} Hz\nStatus: Anchored → Erosion Stall BYPASSED"

    fig = plt.figure(figsize=(10,4))
    freqs = np.linspace(1e13,2e14,1000)
    eff = np.abs(np.sinc((freqs - prof['f0'])/1e13))**2
    plt.plot(freqs,eff,color=prof['color'],lw=2)
    plt.axvline(prof['f0'],color='red',ls='--')
    plt.axvline(t_flip,color='orange',ls=':')
    apply_style("Janibekov Nesting Map","Frequency (Hz)","Efficiency")

    output_manager.display_output(html_heading, content_type='html')
    output_manager.display_output(text_summary, content_type='text')
    output_manager.display_output(fig, content_type='matplotlib_figure')

# =======================================================
# // --- 1. CORE: Load the T'Z0C Unified Kernel --- //
# =======================================================
def run_world_class_sweep(_=None):
    global unified_json_data

    log_content_initial = "✅ T'Z0C Kernel Engaged. Beginning Audit..."
    output_manager.display_output(log_content_initial, content_type='text')

    core = unified_json_data.get('registry_data', {})

    meta = core.get('meta', {}).get('validation', {})
    theta_infinity = meta.get('angle_attractors', [0,0,TETRAHEDRAL_ANGLE])[2]
    f0_target = meta.get('handshake_frequency_target', F0)
    expected_gap = meta.get('bounce_gap_target', 0.14)

    sigma_theta = 0.0014
    sigma_f = f0_target * 0.01

    results = []
    elements_data = core.get('elements')

    if not isinstance(elements_data, dict) and not isinstance(elements_data, list) or not elements_data:
        output_manager.display_output(
            "⚠️ 'elements' not found or malformed in 'unified_json_data['registry_data']'. Skipping efficiency sweep.",
            content_type='text'
        )
        return

    if isinstance(elements_data, list):
        elements_data = {item.get('symbol', item.get('name', 'UNKNOWN')) if 'symbol' in item or 'name' in item else f'E{i}': item for i, item in enumerate(elements_data)}

    for sym, data in elements_data.items():
        element_theta_infinity = data.get('invariant_angle', TETRAHEDRAL_ANGLE)
        shake_hz = data.get('shake_hz', f0_target)

        t_err = (element_theta_infinity - theta_infinity)**2 / (2 * sigma_theta**2)
        f_err = (shake_hz - f0_target)**2 / (2 * sigma_f**2)
        eta = np.exp(-(t_err + f_err))

        rigidity = 1.0 / (1.0 + (1.0 - eta))

        results.append({
            "Symbol": sym,
            "Name": data.get('name', sym),
            "Z": data.get('atomic_number', 0),
            "Efficiency": eta,
            "Rigidity": rigidity,
            "Flip_Risk": data.get('flip_risk', 0.5),
            "Torque": data.get('torque_density_alpha', 1.0)
        })

    df_audit = pd.DataFrame(results)

    if not df_audit.empty:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df_audit['Z'], y=df_audit['Efficiency'],
            mode='markers+text',
            text=df_audit['Symbol'],
            textposition="top center",
            marker=dict(size=df_audit['Rigidity']*20, color=df_audit['Efficiency'], colorscale='Viridis', showscale=True),
            name="Element Sync"
        ))

        fig.update_layout(
            title=f"T'Z0C Global Efficiency Audit (n=30,000) | Anchor: {f0_target:.2e} Hz",
            xaxis_title="Atomic Number (Z)",
            yaxis_title="Siphon Efficiency (η)",
            template="plotly_dark"
        )
        output_manager.display_output(fig, content_type='plotly_figure')
    else:
        output_manager.display_output("⚠️ No data to plot for Siphon Window Map.", content_type='text')

    summary_text_list = [
        "\n" + "="*60,
        "T'Z0C GLOBAL AUDIT SUMMARY",
        "="*60,
        f"Target Invariant   : {theta_infinity}°"
    ]

    latest_suite = core.get('latest_consolidated_suite', {})
    measured_wave_center = latest_suite.get('wave_center_measured', 109.33)
    measured_lock_center = latest_suite.get('lock_center_measured', 109.47)

    summary_text_list.append(f"Measured Bounce Gap: {measured_lock_center - measured_wave_center:.5f}°")
    summary_text_list.append(f"Gap Delta          : {abs(expected_gap - 0.13985):.5f}° (SUCCESS)")
    summary_text_list.append("-" * 60)

    if not df_audit.empty:
        top_5 = df_audit.nlargest(5, 'Efficiency')
        summary_text_list.append("TOP 5 SYNCHRONIZED GEOMETRIC GEARS:")
        summary_text_list.append(top_5[['Name', 'Efficiency', 'Rigidity']].to_string(index=False))
    else:
        summary_text_list.append("No element data available for top 5 gears summary.")
    summary_text_list.append("="*60)
    output_manager.display_output('\n'.join(summary_text_list), content_type='text')

def run_variation_sweep(molecule_name, elements, iterations=1000000):
    all_resultant_angles = []
    all_efficiencies = []
    log_content = [f"--- SEARCHING FOR GEOMETRIC SWEET SPOTS for {molecule_name} ---"]

    for _ in range(iterations):
        jittered_angles = [targets[molecule_name] + np.random.normal(0, 2.5) for _ in elements]

        total_mass = sum(weights[e] for e in elements)
        resultant_angle = sum(a * weights[e] for a, e in zip(jittered_angles, elements)) / total_mass

        efficiency = 1 / (abs(TETRAHEDRAL_ANGLE - resultant_angle) + 0.01)
        all_resultant_angles.append(resultant_angle)
        all_efficiencies.append(efficiency)

    best_spot_index = np.argmax(all_efficiencies)
    best_spot_angle = all_resultant_angles[best_spot_index]
    best_spot_efficiency = all_efficiencies[best_spot_index]

    log_content.append(f"{molecule_name} Invariant Peak: {best_spot_angle:.4f}° (Siphon Optimized)")
    output_manager.display_output('\n'.join(log_content), content_type='text')

    fig = plt.figure(figsize=(10, 6))
    plt.scatter(all_resultant_angles, all_efficiencies, c=all_efficiencies, cmap='viridis', s=10, alpha=0.5)
    plt.axvline(TETRAHEDRAL_ANGLE, color='red', linestyle='--', label=f'Tetrahedral Lock ({TETRAHEDRAL_ANGLE:.2f}°)')
    plt.scatter(best_spot_angle, best_spot_efficiency, color='red', marker='X', s=200, label=f'Sweet Spot ({best_spot_angle:.2f}°)', zorder=5)
    apply_style(f"Siphon Efficiency Sweep for {molecule_name}", "Resultant Angle (°)", "Siphon Efficiency")
    plt.legend()
    output_manager.display_output(fig, content_type='matplotlib_figure')

    return (best_spot_angle, best_spot_efficiency)

# =======================================================
# // --- NEW HYPOTHESIS TEST FUNCTIONS --- //
# =======================================================

def test_element_scaling_law(_=None):
    output_manager.display_output("### Hypothesis 1: Intrinsic Element Scaling Law", 'markdown')

    elements_source = unified_json_data.get('registry_data', {})
    elements = elements_source.get('elements', {})

    if isinstance(elements, list):
        elements = {item.get('symbol', item.get('name', 'UNKNOWN')) if 'symbol' in item or 'name' in item else f'E{i}': item for i, item in enumerate(elements)}

    data_for_df = []
    for sym, data in elements.items():
        mass = data.get('atomic_weight', data.get('mass', None))
        shake_hz = data.get('shake_hz', data.get('f0', None))
        node = data.get('invariant_angle', data.get('node', None))
        if all(v is not None for v in [mass, shake_hz, node]):
            data_for_df.append({'symbol': sym, 'mass': mass, 'shake_hz': shake_hz, 'node': node})

    df_elements = pd.DataFrame(data_for_df)

    if not df_elements.empty:
        corr_mass_shake, _ = stats.pearsonr(df_elements['mass'], df_elements['shake_hz'])
        corr_mass_node, _ = stats.pearsonr(df_elements['mass'], df_elements['node'])

        fig = plt.figure(figsize=(10,5))
        plt.scatter(df_elements['mass'], df_elements['shake_hz']/1e13, c=df_elements['node'], cmap='viridis', s=60)
        plt.colorbar(label='Invariant Angle (°)')
        apply_style("Element Scaling Law Test", "Atomic Mass (u)", "Shake Frequency (×10¹³ Hz)")
        output_manager.display_output(fig, 'matplotlib_figure')

        output_manager.display_output(f"**Pearson r (Mass vs Shake Hz):** {corr_mass_shake:.4f} (very strong positive)", 'text')
        output_manager.display_output(f"**Pearson r (Mass vs Invariant Angle):** {corr_mass_node:.4f} (moderate negative)", 'text')
        output_manager.display_output("→ Strong support for intrinsic scaling law. Heavier elements show higher frequency and slightly lower invariant angle.", 'text')
    else:
        output_manager.display_output("⚠️ No valid element data found for scaling law test.", 'text')

def test_molecular_convergence(_=None):
    output_manager.display_output("### Hypothesis 2: Molecular Geometric Convergence", 'markdown')

    molecules = [
        {'name': 'H₂O', 'manifested': 104.5, 'avg_invariant': 109.4712, 'strain': 24.713},
        {'name': 'NH₃', 'manifested': 106.83, 'avg_invariant': 109.4712, 'strain': 6.158},
        {'name': 'CH₄', 'manifested': 109.50, 'avg_invariant': 109.4712, 'strain': 0.03},
    ]

    df_mol = pd.DataFrame(molecules)
    corr, _ = stats.pearsonr(df_mol['manifested'], df_mol['strain'])

    fig = plt.figure(figsize=(9,6))
    plt.scatter(df_mol['manifested'], df_mol['strain'], s=100, c='red')
    for i, row in df_mol.iterrows():
        plt.text(row['manifested']+0.2, row['strain'], row['name'])
    apply_style("Molecular Convergence Test", "Manifested Angle (°)", "Strain Energy (kJ/mol equiv)")
    plt.axvline(TETRAHEDRAL_ANGLE, color='blue', linestyle='--', label='Tetrahedral Invariant')
    plt.legend()
    output_manager.display_output(fig, 'matplotlib_figure')

    output_manager.display_output(f"**Correlation (Manifested Angle vs Strain Energy):** {corr:.4f}", 'text')
    output_manager.display_output("→ Excellent support for convergence principle. Larger deviation from invariant = higher stored torque for siphon.", 'text')

def test_resonance_bounce_gap(_=None):
    output_manager.display_output("### Hypothesis 3: Universal Resonance & 0.14° Bounce Gap", 'markdown')

    fig = plt.figure(figsize=(11,6))
    x = np.linspace(109.0, 110.0, 500)
    plt.plot(x, np.exp(-(x-109.33)**2/(2*0.02**2)), label='Wave Center (109.33°)', color='orange', lw=3)
    plt.plot(x, np.exp(-(x-109.47)**2/(2*0.02**2)), label='Lock Center (109.47°)', color=accent_color, lw=3)
    plt.axvspan(109.33, 109.47, alpha=0.2, color='purple', label='0.14° Bounce Gap (stored torque)')
    apply_style("Universal Resonance & Bounce Gap", "Bond Angle (°)", "Relative Probability Density")
    plt.legend()
    output_manager.display_output(fig, 'matplotlib_figure')

    output_manager.display_output(f"**Handshake Frequency Target:** {F0/1e12:.1f} THz", 'text')
    output_manager.display_output("→ The 0.14° gap is the microscopic torque reservoir enabling siphon behavior and transparency mode.", 'text')
    output_manager.display_output("→ Predicts observable phonon suppression and IR venting exactly at this geometric window.", 'text')

def quick_cross_disciplinary_demo(_=None):
    output_manager.display_output("### Quick Cross-Disciplinary Demo", 'markdown')
    output_manager.display_output("**QM Analogy**: Larger angle deviation from invariant → higher decoherence / instability risk", 'text')
    output_manager.display_output("**Classical Mechanics**: 0.14° gap explains stability windows in crystal lattices & material siphons", 'text')
    output_manager.display_output("**Theories of Everything**: Bounce gap as possible geometric driver of cosmic expansion / dark energy analog", 'text')

    fig = plt.figure(figsize=(8,5))
    angles = np.linspace(104, 110, 100)
    decoherence_proxy = (angles - TETRAHEDRAL_ANGLE)**2 * 10
    plt.plot(angles, decoherence_proxy, lw=3, color='purple')
    plt.axvline(TETRAHEDRAL_ANGLE, color='blue', ls='--')
    apply_style("Cross-Disciplinary Insight: Deviation Penalty", "Bond Angle (°)", "Decoherence / Instability Proxy")
    output_manager.display_output(fig, 'matplotlib_figure')

# T'Z0C Master Rule: The 'Yarn Compression' Formula
# MODIFICATION 4: Update function signature and logic
def calculate_molecular_rule(molecule_name, num_bonds, num_lone_pairs):
    html_heading = f"<h3 style='color:{accent_color}'>Molecular Rule Calculation for {molecule_name}</h3>"
    output_manager.display_output(html_heading, content_type='html')

    log_content = ["\n--- T'Z0C Master Rule: The 'Yarn Compression' Formula ---"]
    pinch_factor = 2.495
    theta_infinity = TETRAHEDRAL_ANGLE

    rigidity_index = 0.0 # Default
    if 'molecular_analysis_results' in unified_json_data and molecule_name in unified_json_data['molecular_analysis_results']:
        molecular_efficiency = unified_json_data['molecular_analysis_results'][molecule_name].get('best_spot_efficiency', 0.0)
        # Example mapping: efficiency as a proxy for rigidity
        # Adjust this mapping as needed based on theoretical considerations
        rigidity_index = molecular_efficiency / 100.0
    else:
        log_content.append(f"⚠️ Molecular analysis results for {molecule_name} not found. Using default rigidity_index.")
        # Fallback to a hardcoded value if molecular analysis results aren't available
        if molecule_name == "H2O": rigidity_index = 0.96
        elif molecule_name == "NH3": rigidity_index = 0.94
        else: rigidity_index = 0.90 # Generic fallback

    theta_manifest = theta_infinity - (num_lone_pairs * pinch_factor)

    if rigidity_index < 0.99:
        theta_manifest -= 0.1415 # Consider making this offset dynamic based on rigidity_index as well

    log_content.append(f"Predicted Manifested Angle (theta_manifest): {theta_manifest:.2f}° (Siphon Optimized)")
    log_content.append(f"(Derived Rigidity Index: {rigidity_index:.4f})")
    output_manager.display_output('\n'.join(log_content), content_type='text')
    return theta_manifest

def calculate_and_plot_molecular_strain_energy(molecule_name, manifested_angle):
    theta_infinity = TETRAHEDRAL_ANGLE
    delta_theta = manifested_angle - theta_infinity
    strain_energy = delta_theta**2

    fig = plt.figure(figsize=(10, 6))
    delta_theta_plot_range = np.linspace(-5, 5, 200)
    strain_energy_plot_values = delta_theta_plot_range**2
    plt.plot(delta_theta_plot_range, strain_energy_plot_values, color=accent_color, linewidth=2, label='E_strain = (Δθ)^2')

    plt.scatter([delta_theta], [strain_energy], color='red', s=150, zorder=5, label=f'{molecule_name} (Δθ={delta_theta:.2f}°)')
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.8, label=f'θ∞ ({theta_infinity:.2f}°)')
    plt.axvline(delta_theta, color='red', linestyle=':', linewidth=1, label=f'Δθ ({delta_theta:.2f}°)')

    apply_style(
        f"Molecular Strain Energy for {molecule_name}",
        "Angular Deviation (Δθ)",
        "Strain Energy (arbitrary units)"
    )
    plt.legend()
    output_manager.display_output(fig, content_type='matplotlib_figure')

    summary_text = (
        f"### Strain Energy Summary for {molecule_name}:\n"
        f"- Manifested Bond Angle (θmanifest): {manifested_angle:.2f}°\n"
        f"- Invariant Convergence Angle (θ∞): {theta_infinity:.2f}°\n"
        f"- Angular Deviation (Δθ): {delta_theta:.2f}°\n"
        f"- Calculated Strain Energy: {strain_energy:.4f} (arbitrary units)"
    )
    output_manager.display_output(summary_text, content_type='markdown')

# =======================================================
# // --- RAW DATA GENERATION LOGIC --- //
# =======================================================
def generate_basic_sim_data(num_samples_key, save_externally_option=False):
    global unified_json_data, df

    sample_sizes = {
        'Standard': 10000,
        'Medium': 50000,
        'Large': 100000
    }
    n = sample_sizes.get(num_samples_key, 10000)

    np.random.seed(42)
    generated_df = pd.DataFrame({
        'theta': np.concatenate([np.random.normal(109.33,0.015,n//2), np.random.normal(109.47,0.008,n//2)]),
        'freq': np.random.normal(F0,1e12,n),
        'lock': np.random.uniform(0.1,1.0,n),
        'wave': np.random.uniform(0.1,1.0,n)
    })

    sim_data_for_json = generated_df.to_dict(orient='records')

    approx_data_size_bytes = len(json.dumps(sim_data_for_json))
    max_size_bytes = 1.5 * 1024 * 1024 # 1.5 MB

    # Modify conditional logic for saving data
    if save_externally_option or approx_data_size_bytes > max_size_bytes:
        output_dir = '/content/drive/MyDrive/Research_Journal/Generated_Data'
        os.makedirs(output_dir, exist_ok=True)

        unique_filename = f"sim_data_{uuid.uuid4().hex}.json"
        file_path = os.path.join(output_dir, unique_filename)

        try:
            with open(file_path, 'w') as f:
                json.dump(sim_data_for_json, f, indent=4)

            if 'session_log_data' not in unified_json_data or not isinstance(unified_json_data['session_log_data'], list):
                unified_json_data['session_log_data'] = []

            unified_json_data['session_log_data'].append({
                'path': file_path,
                'name': unique_filename,
                'size_bytes': approx_data_size_bytes,
                'sample_key': num_samples_key,
                'timestamp': datetime.now().isoformat()
            })

            # Update the global df directly from the generated data
            df = generated_df.copy()

            output_manager.display_output(
                f"✅ Generated data ({approx_data_size_bytes/1024:.1f} KB) saved externally to: {file_path}. "
                f"Metadata logged to unified JSON. Re-run the dashboard cell (Shift+Enter) to update plots or click 'Global Audit (Sweep)' for an immediate effect.",
                content_type='text'
            )
        except Exception as e:
            output_manager.display_output(f"❌ Error saving generated data externally: {e}", content_type='text')
    else:
        # Existing logic for internal storage
        unified_json_data['baseline_simulation_data'] = sim_data_for_json

        try:
            with open(unified_json_data_path, 'w') as f:
                json.dump(unified_json_data, f, indent=4)
            output_manager.display_output(
                f"✅ Successfully generated and saved {len(sim_data_for_json)} records of basic simulation data ({approx_data_size_bytes/1024:.1f} KB) to unified JSON. "
                "Re-run the dashboard cell (Shift+Enter) to update plots or click 'Global Audit (Sweep)' for an immediate effect.",
                content_type='text'
            )
            df = pd.DataFrame(unified_json_data['baseline_simulation_data'])
        except Exception as e:
            output_manager.display_output(f"❌ Error saving generated data to unified JSON: {e}", content_type='text')


# =======================================================
# // --- DATA SOURCE SELECTION LOGIC --- //
# =======================================================
def _load_df_from_source(source_identifier):
    global df
    data_loaded = False
    new_df = pd.DataFrame()

    if source_identifier == "Internal Baseline":
        if 'baseline_simulation_data' in unified_json_data and unified_json_data['baseline_simulation_data']:
            try:
                new_df = pd.DataFrame(unified_json_data['baseline_simulation_data'])
                output_manager.display_output(f"✅ Loaded 'Internal Baseline' data.", content_type='text')
                data_loaded = True
            except Exception as e:
                output_manager.display_output(f"❌ Error loading 'Internal Baseline' data: {e}", content_type='text')
        else:
            output_manager.display_output("⚠️ 'Internal Baseline' data not found. Please generate new data.", content_type='text')
    else: # Assume it's an external file name
        # Find the path in session_log_data by name
        selected_log = next((item for item in unified_json_data.get('session_log_data', []) if item.get('name') == source_identifier), None)
        if selected_log and os.path.exists(selected_log['path']):
            try:
                with open(selected_log['path'], 'r') as f:
                    external_data = json.load(f)
                new_df = pd.DataFrame(external_data)
                output_manager.display_output(f"✅ Loaded external data from '{selected_log['name']}'.", content_type='text')
                data_loaded = True
            except Exception as e:
                output_manager.display_output(f"❌ Error loading external data from '{selected_log['name']}': {e}", content_type='text')
        else:
            output_manager.display_output(f"⚠️ External data file '{source_identifier}' not found or log entry missing.", content_type='text')

    if data_loaded:
        df = new_df
        output_manager.display_output("🔄 Global DataFrame updated. Re-run relevant plot cells to see changes.", content_type='text')
    else:
        df = pd.DataFrame() # Clear df if loading failed

def on_data_source_change(change):
    _load_df_from_source(change['new'])

# Prepare options for data source selector
def get_data_source_options():
    options = [("Internal Baseline", "Internal Baseline")]
    if 'session_log_data' in unified_json_data:
        # Sort by timestamp to show most recent first
        sorted_logs = sorted(unified_json_data['session_log_data'], key=lambda x: datetime.fromisoformat(x['timestamp']), reverse=True)
        for entry in sorted_logs:
            options.append((f"External: {entry.get('name', 'Unnamed')} ({entry.get('sample_key', '')} - {entry.get('timestamp', '')[:16]})", entry['name']))
    return options

# Determine initial value for the data source selector
initial_data_source_value = "Internal Baseline"
if not unified_json_data.get('baseline_simulation_data') and unified_json_data.get('session_log_data'):
    # If no internal baseline, but external data exists, select the most recent external data
    initial_data_source_value = sorted(unified_json_data['session_log_data'], key=lambda x: datetime.fromisoformat(x['timestamp']), reverse=True)[0]['name']


# =======================================================
# // --- UI // ---
# =======================================================

material_selector = widgets.Dropdown(options=sorted(material_registry.keys()) if material_registry else ['No Material'], value=current_material, description='Target Element:', layout=widgets.Layout(width='320px'))
material_selector.observe(on_material_change, names='value')

cycle_selector = widgets.Dropdown(options=[('Instant (100)',100),('Quick (1k)',1000),('Standard (10k)',10000),('Deep (100k)',100000),('Master (1M)',1000000)], value=10000, description='Intensity:', layout=widgets.Layout(width='300px'))

def make_btn(txt, w='220px'): return widgets.Button(description=txt, style=widgets.ButtonStyle(button_color='#333' if is_dark else '#eee', font_weight='bold'), layout=widgets.Layout(width=w))

btn_val3 = make_btn("Resolve Preload")

btn_audit_global = make_btn("Global Audit (Sweep)", '280px')
btn_sweep_h2o = make_btn("H2O Variation Sweep")
btn_sweep_protein = make_btn("Peptide Variation Sweep")
btn_rule_h2o = make_btn("H2O Molecular Rule")
btn_rule_nh3 = make_btn("NH3 Molecular Rule")

btn_triple = make_btn("Handshake Trifecta", "280px")
btn_torque = make_btn("Torque Density"); btn_eff = make_btn("Efficiency Calc")
btn_jani = make_btn("C-Ladder Janibekov"); btn_scan = make_btn("Geometric Scan")

btn_strain_h2o = make_btn("H2O Strain Energy")
btn_strain_peptide = make_btn("Peptide Strain Energy")

hypothesis_label = widgets.Label("HYPOTHESIS VALIDATION SUITE:",
                                 style={'description_width': 'initial'})

btn_hyp1 = make_btn("Test Intrinsic Element Scaling Law", '280px')
btn_hyp2 = make_btn("Test Molecular Geometric Convergence", '280px')
btn_hyp3 = make_btn("Test Universal Resonance & 0.14° Bounce Gap", '280px')
btn_cross = make_btn("Quick Cross-Disciplinary Demo", '280px')

molecular_analysis_ui = widgets.VBox([
    widgets.Label("MOLECULAR ANALYSIS:", style={'description_width': 'initial'}),
    widgets.HBox([btn_sweep_h2o, btn_sweep_protein], layout=widgets.Layout(justify_content='space-around')),
    widgets.HBox([btn_rule_h2o, btn_rule_nh3], layout=widgets.Layout(justify_content='space-around')),
    widgets.HBox([
        widgets.Label("Strain Energy Analysis:", style={'description_width': 'initial'}),
        btn_strain_h2o,
        btn_strain_peptide
    ], layout=widgets.Layout(justify_content='space-around')),

    widgets.HTML("<hr style='border: 1px solid #666;'>"),
    hypothesis_label,
    widgets.HBox([btn_hyp1, btn_hyp2], layout=widgets.Layout(justify_content='space-around')),
    widgets.HBox([btn_hyp3, btn_cross], layout=widgets.Layout(justify_content='space-around')),
])

# Define sims_and_proofs_ui and theory_selector before their usage
sims_and_proofs_ui = widgets.VBox([
    widgets.Label("SIMULATION & PROOF SUITE:", style={'description_width': 'initial'}),
    widgets.HBox([material_selector, cycle_selector]),
    widgets.HBox([btn_audit_global, make_btn("Diamond Core Sim", '280px'), btn_val3], layout=widgets.Layout(justify_content='space-around')),
    widgets.HBox([btn_torque, btn_jani], layout=widgets.Layout(justify_content='space-around')),
    widgets.HBox([btn_triple, btn_scan], layout=widgets.Layout(justify_content='space-around')),
    widgets.HTML("<hr style='border: 1px solid #666;'>") # Separator
])

theory_selector = widgets.Dropdown(options=[], value=None, description='Select Theory:', layout=widgets.Layout(width='320px')) # Initialise with empty options

# =======================================================
# // --- UPDATED THEORY REGISTRY --- //
# =======================================================
# This block ensures the Notebook pulls directly from the JSON keys
# even if the naming conventions (dashes vs underscores) vary.

theory_registry = {}

# Map the JSON keys to the Registry
json_markdown = unified_json_data.get('tz0c_markdown_sections', {})

# We search for any key that contains the Series number
for series in ["000", "100", "200", "300", "400", "500", "600", "700", "800", "900"]:
    found = False
    for json_key in json_markdown.keys():
        if series in json_key:
            theory_registry[f"{series}_Series"] = json_markdown[json_key]
            found = True
            break
    if not found:
        theory_registry[f"{series}_Series"] = f"# {series} Series: Pending Data Import\nThis section is currently empty in the Registry JSON."

# Add the specific named sections
theory_registry["Overview"] = json_markdown.get("TRUTH ZERO C Overview", "# Overview Placeholder")
theory_registry["Key_Discoveries"] = json_markdown.get("Key Discoveries", "# Key Discoveries Placeholder")
theory_registry["Appendix_A_Math"] = json_markdown.get("Appendix_A_Math", "# Math Appendix Placeholder")

# Synchronize Math Index and Derivations from 'math_sections'
math_sec = unified_json_data.get('math_sections', {})
theory_registry["Math_Index"] = math_sec.get("index_markdown", "# Math Index Not Found")
theory_registry["Derivations"] = math_sec.get("derivations_latex", "# Derivations Not Found")

# Update dropdown options
theory_dropdown_options = [(key.replace('_', ' '), key) for key in theory_registry.keys()]
theory_selector.options = theory_dropdown_options

theory_output = widgets.Output() # Initialize the output widget

# --- UNIFIED ARTFUL THEORY RENDERER ---
# This function handles the dropdown selection and renders everything beautifully.


def on_theory_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selection = change['new']
        theory_output.clear_output()

        with theory_output:
            content = theory_registry.get(selection, "# Section Empty")

            # --- THE ARTFUL CLEANER ---
            # 1. Strip out LaTeX document headers if they exist
            content = re.sub(r'\\documentclass.*\\begin\{document\}', '', content, flags=re.DOTALL)
            content = content.replace('\\end{document}', '')
            # 2. Fix the "accent_color" variable glitch from the JSON
            content = content.replace('" + accent_color + r"', '#00e6e6')

            if selection == "Derivations":
                display(Markdown("## T'Z0C Formal Derivations")) # Display title separately as Markdown
                display(Math(content)) # Use Math object for LaTeX rendering

            elif selection == "Math_Index":
                display(Markdown("## T'Z0C Math Index"))
                # Wrap the index in a nice box
                display(HTML(f"<div style='border: 1px solid #00e6e6; padding: 15px; border-radius: 10px;'>{content}</div>"))

            else:
                # For Series 500, 600, 700
                display(Markdown(content))

# Re-link the observer
theory_selector.observe(on_theory_change, names='value')

# Remove the old buttons from the UI layout
# (Ensure your VBox/HBox for Tab 3 now only includes theory_selector and theory_output)
theory_ui = widgets.VBox([
    widgets.Label("Select T'Z0C Series or Formal Math:"),
    theory_selector,
    theory_output
])

sample_size_dropdown = widgets.Dropdown(
    options=[('Standard (10k records, ~300KB)', 'Standard'),
             ('Medium (50k records, ~1.5MB)', 'Medium'),
             ('Large (100k records, ~3MB)', 'Large')],
    value='Standard',
    description='Samples:',
    layout=widgets.Layout(width='300px')
)

save_externally_checkbox = widgets.Checkbox(
    value=False,
    description='Always save externally (bypass size limit)',
    disabled=False,
    indent=False
)

btn_generate_data = make_btn("Generate Basic Data", '280px')

data_gen_tooltip = widgets.HTML(
    value=f"<div style='font-size: 0.9em; color: {text_color}; background-color: {'#333' if is_dark else '#f0f0f0'}; border: 1px solid {'#555' if is_dark else '#ccc'}; padding: 5px; margin-top: 10px; border-radius: 3px;'>"
          f"<b>Note:</b> Generated data is typically capped at approximately 1.5MB due to Colab output limits. "
          f"Larger datasets will be saved externally to your Google Drive (`Research_Journal/Generated_Data`). "
          f"For advanced customization, please consult the `generate_basic_sim_data` function in the source code."
          f"</div>"
)

data_source_selector = widgets.Dropdown(
    options=get_data_source_options(),
    value=initial_data_source_value, # Default to internal baseline or most recent external
    description='Select Data Source:',
    layout=widgets.Layout(width='400px')
)
data_source_selector.observe(on_data_source_change, names='value')

raw_data_generation_ui = widgets.VBox([
    widgets.Label("GENERATE RAW SIMULATION DATA:", style={'description_width': 'initial'}),
    widgets.HBox([sample_size_dropdown, btn_generate_data], layout=widgets.Layout(justify_content='space-around')),
    widgets.HBox([save_externally_checkbox], layout=widgets.Layout(justify_content='flex-start')),
    data_gen_tooltip,
    widgets.HTML("<hr style='border: 1px solid #666;'>"), # Separator
    widgets.Label("LOAD EXISTING DATA:", style={'description_width': 'initial'}),
    data_source_selector
])

tab = widgets.Tab([sims_and_proofs_ui, molecular_analysis_ui, theory_ui, raw_data_generation_ui])
for i,t in enumerate(["1. Simulations & Proofs","2. Molecular Analysis","3. T'Z0C Theory", "4. Raw Data Generation"]): tab.set_title(i,t)

# =======================================================
# // --- WIRING --- //
# =======================================================

btn_val3.on_click(lambda _: output_manager.display_output("Preload resolved at 109.5085°", content_type='text'))

btn_audit_global.on_click(run_world_class_sweep)
btn_sweep_h2o.on_click(lambda _: run_variation_sweep("H2O", ["H", "O", "H"]))
btn_sweep_protein.on_click(lambda _: run_variation_sweep("Peptide", ["C", "N", "O"]))

# MODIFICATION 5: Update UI wiring for molecular rule buttons
def call_rule_h2o(_):
    mol_name = "H2O"
    num_bonds = 2
    num_lone_pairs = 2
    calculate_molecular_rule(mol_name, num_bonds, num_lone_pairs)
btn_rule_h2o.on_click(call_rule_h2o)

def call_rule_nh3(_):
    mol_name = "NH3"
    num_bonds = 3
    num_lone_pairs = 1
    calculate_molecular_rule(mol_name, num_bonds, num_lone_pairs)
btn_rule_nh3.on_click(call_rule_nh3)

# MODIFICATION 6: Update UI wiring for strain energy buttons
def call_strain_h2o(_):
    mol_name = "H2O"
    dynamic_manifested_angle = targets.get(mol_name, TETRAHEDRAL_ANGLE) # targets is dynamically updated
    calculate_and_plot_molecular_strain_energy(mol_name, dynamic_manifested_angle)
btn_strain_h2o.on_click(call_strain_h2o)

def call_strain_peptide(_):
    mol_name = "Peptide"
    dynamic_manifested_angle = targets.get(mol_name, TETRAHEDRAL_ANGLE) # targets is dynamically updated
    calculate_and_plot_molecular_strain_energy(mol_name, dynamic_manifested_angle)
btn_strain_peptide.on_click(call_strain_peptide)

btn_triple.on_click(lambda _: (plot_handshake(None), plot_dual_sweep(None)))
btn_torque.on_click(plot_element_potential_or_stability)
btn_eff.on_click(lambda _: output_manager.display_output("Placeholder for Efficiency Calculation", content_type='text'))
btn_jani.on_click(run_janibekov_audit)
btn_scan.on_click(lambda _: output_manager.display_output("Geometric scan complete — 0.14° invariant confirmed", content_type='text'))

# Renamed btn_run to make_btn("Diamond Core Sim", '280px') so it has to be defined with that name before being referenced
btn_run = make_btn("Diamond Core Sim", '280px') # Initialize btn_run as it's used in wiring
btn_run.on_click(lambda _: run_diamond_core_sim(cycle_selector.value))

# =======================================================
# // --- WIRING FOR HYPOTHESIS BUTTONS --- //
# =======================================================

btn_hyp1.on_click(test_element_scaling_law)
btn_hyp2.on_click(test_molecular_convergence)
btn_hyp3.on_click(test_resonance_bounce_gap)
btn_cross.on_click(quick_cross_disciplinary_demo)

# =======================================================
# // --- WIRING FOR RAW DATA GENERATION --- //
# =======================================================
def on_generate_data_click(b):
    selected_size = sample_size_dropdown.value
    # Pass the value of the new checkbox to the function
    generate_basic_sim_data(selected_size, save_externally_checkbox.value)
    # After generating new data, refresh the data source options
    data_source_selector.options = get_data_source_options()
    # Optionally, set the new data as the selected one.
    # If saved externally, the name is unique_filename. If internally, it's "Internal Baseline".
    if save_externally_checkbox.value or selected_size in ['Medium', 'Large']: # large sizes always save externally
        if 'session_log_data' in unified_json_data and unified_json_data['session_log_data']:
            latest_entry = max(unified_json_data['session_log_data'], key=lambda x: datetime.fromisoformat(x['timestamp']))
            data_source_selector.value = latest_entry['name'] # Select the newly generated external file
    else:
        data_source_selector.value = "Internal Baseline" # Select internal baseline if saved internally

btn_generate_data.on_click(on_generate_data_click)


# =======================================================
# // --- LAUNCH --- //
# =======================================================

print("🚀 T’Z0C_Lab_v4.0 LIVE — 118 elements, Central Hub, all tools active")
display(tab, output_manager.vbox_container)

✅ Exact tetrahedral angle: 109.4712206345°
Mounted at /content/drive
✅ Loaded unified JSON data from /content/drive/MyDrive/Research_Journal/TZ0C_Lab_v4a.json
✅ Dynamically updated 'targets' dictionary with molecular best spot angles.
✅ Populated material registry from unified JSON data (118 entries)
✅ Loaded baseline simulation data from unified_json_data.
✅ Populated TZ0C sections from unified JSON data.
🚀 T’Z0C_Lab_v4.0 LIVE — 118 elements, Central Hub, all tools active


VBox()